In [1]:
import torch
import torchvision
from torch import nn
from torch.utils import data
from torchvision import transforms
import numpy as np

## CNN's

ConvNets exploit natural image structure, reduce data in latent space without computationally intractable operations by aggregation.

**Spatial Invariance** - sweep an image by patches to learn representation of each subpatch of image, then aggregate a representation. (Aggregation of subspace).


### 2 Major Criteria to make this suitable for Computer vision:


**Translation Invariance** - Each Region should be initially treated indiscriminantly (we attempt to learn an identical thing about each part of the image initially).

**Locality** - We focus on subregions of the image to begin with, without concern for how these subregions will affect other distant regions until later layers when the information is aggregated.

### Criteria Application in ConvNets

In a ConvNet, we take a weighted function of pixles within a locality. We place the result of this output in the corresponding output subsection, AKA corner aggregation maintains corner status, thus maintaining the concept of spatiality.

Translational Invariance is maintained by keeping the weight filter constant, such that no matter where we are in the image we are maintaining the same "Question" / learning an identical thing about each part of the image initially.

Locality is maintained by having a convolutional filter which is less than the size of the image, thus aggregating information along various localities (subregions). AKA if v is the **kernel** / **filter** (this is an abstraction of a Kernel seen in Kernel Density, which is a weighting function for nearby points / features valuing them by distances, with a Neural Kernel instead valuing them by placement) then v > index i for some i will be 0 (far away features not weighted at all as do not impact local subimage understanding, just as window in KDE is capped by lambda

Locality allows us to shrink our filters below the size of the image and learn local filters. Transitional invariance allows us to use one such filter uniformly across the image indiscriminantly. These together combine to the idea that we can learn a homogeneous feature about each locality, and by aggregating such homogenous questions can learn a valuable latent representation that can be used to classify an image. This methodology is extremely parameter efficient, however, as we only need to learn one weighting function per layer, drastically reducing the combinatorial parameterization of our models to a few hundred

### Why is it called a Convolution?

Convolution is probability joint distribution adds up to something which is = to integral of a = x and b = z-x for across all x. This is equivalent to a sliding window across x in the discrete sense, because the operation in and of itself does not look at joint probability but does weigh possibilities in a convolutional fashion. In the 2 dim sense: x(a,b) * y(z-a, w-b) is a sliding window across the entirety of the [0,z]x[0,w] possibility space, attempting all combinatorial possibilities discretely. That is a rough discrete mapping to a convolution, where we aggregate all joint possibilities to understand the joint probability of an outcome of two variables. Also use one function to scale the other with an AND clause, just as is done in a convoluted multiplication

In practice the mapping of the sliding filter is actually 3 dimensional to account for a third dimension: color in the RGB space

Fundamental Operation: We convolve a weighting filter across an image to obtain an aggregation of several localities

## Convolutions on Images

Operation is far closer to cross correlation (dot product) as aggregating how frequent our valuable features are. If intensity in a certain position is given a higher weight and it is more intense, then we will encode a greater number of logits of information (a valuable feature for classification present there, can map to ranges (37+ = presence of features => cat) This cross correlation is implemented below

In [2]:
def cross_cor(t1: torch.Tensor, kernel: torch.Tensor) -> torch.Tensor:
    """Applies a kernel filter across a tensor, producing an aggregated latent representation"""
    # New shape is how far short of endpoint filter must stop (because it cannot exceed boundaries of image) + 1 for the stopping iteration (if it is sized 2, it still aggregates when 2 away from edge)
    new_shape = t1.shape[0] - kernel.shape[0] + 1, t1.shape[1] - kernel.shape[1] + 1
    # A latent representation of aggregated localities
    latent_representation = torch.rand(new_shape[0], new_shape[1]) # Using image dimensions
    # Iterating and cross correlating
    for i in range(new_shape[0]):
        for j in range(new_shape[1]):
            # Will dot tensors together via multiplication operator - this is the default
            # Obtaining locality, using stride of 1, of sized kernel. Convoluted along
            inter_tensor = t1[i:i+ kernel.shape[0], j:j+kernel.shape[1]]
            # Obtaining cross correlation of locality with kernel (using kernel to weigh sum of information in locality)
            weighted_locality_rep = (inter_tensor * kernel).sum() # Aggregating weighted feature information (reduces latent space rather than just weights)
            latent_representation[i][j] = weighted_locality_rep
    return latent_representation
            
    

In [3]:
cross_cor(torch.randn(4,3), torch.randn(2,2)) # Latent representation maintains aggregated features in localities

tensor([[ 3.1546, -1.8314],
        [ 1.0682, -2.0010],
        [-0.0398, -1.3013]])

In [4]:
def cross_cor_4d(t1: torch.Tensor, kernel: torch.Tensor) -> torch.Tensor:
    """Applies a kernel filter across a tensor, producing an aggregated latent representation"""
    # New shape is how far short of endpoint filter must stop (because it cannot exceed boundaries of image) + 1 for the stopping iteration (if it is sized 2, it still aggregates when 2 away from edge)
    new_shape = t1.shape[0] - kernel.shape[0] + 1, t1.shape[1] - kernel.shape[1] + 1, t1.shape[2] - kernel.shape[2] + 1, t1.shape[3] - kernel.shape[3] 
    # A latent representation of aggregated localities
    latent_representation = torch.rand(new_shape[0], new_shape[1], new_shape[2], new_shape[3]) # Using image dimensions
    # Iterating and cross correlating
    for i in range(new_shape[0]):
        for j in range(new_shape[1]):
            for k in range(t1.shape[1]):
                # Will dot tensors together via multiplication operator - this is the default
                # Obtaining locality, using stride of 1, of sized kernel. Convoluted along
                inter_tensor = t1[i:i+ kernel.shape[0], j:j+kernel.shape[1], k:k+kernel.shape[2], 0:kernel.shape[3]]
                # Obtaining cross correlation of locality with kernel (using kernel to weigh sum of information in locality)
                weighted_locality_rep = (inter_tensor * kernel).sum() # Aggregating weighted feature information (reduces latent space rather than just weights)
                latent_representation[i][j][k] = weighted_locality_rep
    return latent_representation

In [5]:
# Defining convolutional layer
class Conv2D(torch.nn.Module):
    def __init__(self, kernel_size: tuple):
        super().__init__()
        # These will be a learnable parameter. The NN will learn what it should consider important
        self.weights = torch.nn.Parameter(torch.randn(kernel_size))
        self.bias = torch.nn.Parameter(torch.zeros(1))
        self.weights = torch.nn.init.xavier_uniform_(self.weights)
        
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        return torch.nn.functional.relu(cross_cor(X, self.weights) + self.bias)

    
class Flatten(torch.nn.Module):
    """Non-Sequential flatten"""
    
    def __init__(self):
        super().__init__()
    
    def forward(self, inp: torch.Tensor)->torch.Tensor:
        temp = inp.detach().numpy().flatten()
        new_tensor = torch.Tensor(temp)
        new_tensor.requires_grad_ = True
        return new_tensor
        
    
    

In [6]:
class BasicConvNet(torch.nn.Module):
    def __init__(self, *kernel_sizes):
        super().__init__()
        self._modules["layer_1"] = Conv2D(kernel_sizes[0])
        self._modules["layer_2"] = Conv2D(kernel_sizes[1])
        self._modules["layer_3"] = Flatten()
        self._modules["layer_4"] = torch.nn.Linear(100, 10)
    
    def forward(self, X: torch.Tensor)-> torch.Tensor:
        # Applying both convolutions
        X = self._modules["layer_1"](X)
        X = self._modules["layer_2"](X)
        X = self._modules["layer_3"](X)
        X = self._modules["layer_4"](X)
        return X

In [7]:
import torch
import torchvision
from torch import nn
from torch.utils import data
from torchvision import transforms

In [8]:
# Function from previous notebook
# Converting to function for future use, default num_workers is 4 bc CPU threads
def load_fashion_mnist(batch_size: int = 512, num_workers: int = 4):
    data_transform = transforms.ToTensor() # Obtaining data to tensor converter
    
    # Downloading data
    mnist_train = torchvision.datasets.FashionMNIST(root = "../data", train = True, transform = data_transform, download= True)  # Defining fashion MNIST train from torch datasets
    mnist_test = torchvision.datasets.FashionMNIST(root = "../data", train = False, transform = data_transform, download = True)
    
    # Loading data onto an iterator
    train_data_loader = data.DataLoader(mnist_train, batch_size, shuffle = True, num_workers = 4)
    test_data_loader = data.DataLoader(mnist_test, batch_size, shuffle = True, num_workers = 4)
    
    # Returning iterator
    return train_data_loader, test_data_loader 
    

In [9]:
train_loader, test_loader = load_fashion_mnist(256, 4)

/Users/gilpasternak/Library/Python/3.8/lib/python/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [10]:
# Applying init to model to initialize all layer weights
def build_model():
    # Kernels are iterating across all three dimensions and all batch sizes simultaneously
    model = BasicConvNet((15,15,1,1), (5,5,1,1))
    trainer = torch.optim.Adam(model.parameters(), lr = 0.05, weight_decay=0.05)
    loss = torch.nn.CrossEntropyLoss()
    return model, trainer, loss

In [11]:
model, trainer, loss = build_model()

In [12]:
# Note: 4D code is merely an outline

In [13]:
model(torch.randn(28,28)) # 2 dimensional ConvNet Functional

tensor([ 392.8381,  763.1307, -229.3624, -270.9319,   52.6995, -673.6146,
         465.8849, -510.0398,  188.2624,  -13.3621], grad_fn=<AddBackward0>)

In [14]:
class ConvNet(torch.nn.Module):
    def __init__(self, *dims):
        # Weigh + aggregate information twice by convolving it with a localized weighting kernel, then map to output space
        # Call to parent constructor initializes modules
        super().__init__()
        self._modules["layer_1"] = torch.nn.Conv2d(1,1, kernel_size = (19,19))
        self._modules["layer_2"] = torch.nn.Conv2d(1,1, kernel_size = (6,6)) 
        self._modules["remaining_network"] = torch.nn.Sequential(torch.nn.Flatten(), torch.nn.Linear(100,10), torch.nn.Softmax())
        
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """Instance method defining computational mapping to output space"""
        X = torch.nn.functional.relu(self._modules["layer_1"](X))
        #X = torch.nn.functional.relu(self._modules["layer_2"](X))
        # Convolving, breaking linearity, flattening, mapping to output space
        X = self._modules["remaining_network"](X)
        # Returning result
        return X
                                                                

In [15]:
# 1-1+1, 1-1+1, 5-2+1, 5-1+1 stride 1 => output only loses a single column
torch.nn.Conv2d(1,1, kernel_size = (2,1), bias = False)(torch.rand((1,1, 5,5)))

tensor([[[[-0.0010,  0.1327,  0.0783,  0.0569,  0.2965],
          [ 0.0611,  0.1564,  0.2840,  0.0636,  0.1254],
          [ 0.0887,  0.3027, -0.0353,  0.1133,  0.2003],
          [ 0.1212,  0.1300,  0.3185,  0.1228,  0.1898]]]],
       grad_fn=<ThnnConv2DBackward>)

In [16]:
# Checking if convolution can be performed on Fashion MNIST data
for batch, label in train_loader:
    convoluted = torch.nn.Conv2d(1,1, kernel_size = (19,19), bias = False)(batch)
    print(convoluted)
   

tensor([[[[-1.4686e-01, -2.3013e-01, -2.4164e-01,  ..., -5.9070e-02,
           -2.9749e-02, -5.1496e-02],
          [ 6.5166e-02,  7.6217e-02, -6.9952e-02,  ...,  5.6009e-02,
            1.6886e-01,  2.3682e-01],
          [ 2.2208e-01,  2.2194e-01,  9.8507e-02,  ...,  1.0168e-01,
            5.2094e-02,  4.1981e-02],
          ...,
          [-9.4679e-02, -3.5465e-02,  5.0455e-02,  ...,  7.3530e-02,
            2.3674e-02, -4.4289e-04],
          [ 1.5935e-01,  1.7247e-01,  1.1831e-01,  ...,  6.1787e-03,
           -6.3479e-02, -9.1474e-02],
          [ 1.4097e-01,  7.1105e-02,  2.9491e-02,  ..., -6.9407e-02,
           -5.0927e-02, -4.0624e-02]]],


        [[[ 1.1797e-01,  6.9823e-02, -2.3049e-02,  ..., -1.8830e-02,
           -6.8329e-03, -8.8223e-02],
          [ 1.4536e-01,  3.3952e-02, -1.0337e-01,  ..., -4.1437e-02,
           -7.1317e-03, -8.0455e-02],
          [ 6.6670e-02, -2.8716e-02, -7.8999e-02,  ..., -1.2308e-01,
            5.8400e-02, -1.2957e-01],
          ...,
   

tensor([[[[ 1.0125e-01,  1.5130e-01,  8.2832e-02,  ..., -2.2153e-02,
            7.1236e-02,  1.7393e-01],
          [ 1.2893e-01,  1.1683e-01,  5.4842e-02,  ...,  1.0888e-02,
            1.1950e-01,  1.9858e-01],
          [ 7.7019e-02,  4.6416e-02,  1.7370e-01,  ...,  5.8233e-03,
            1.9788e-01,  1.9547e-01],
          ...,
          [-8.0992e-02,  5.2292e-02, -2.0273e-02,  ...,  6.3293e-02,
            1.0956e-01,  1.5540e-01],
          [-1.3142e-01,  3.6483e-02, -2.7856e-02,  ...,  1.8772e-02,
            1.4623e-01,  1.9842e-01],
          [-2.0819e-01, -1.4337e-01, -2.4337e-01,  ..., -2.1599e-02,
            6.6687e-02,  7.7809e-02]]],


        [[[-8.8142e-02, -6.8656e-02,  2.0044e-01,  ...,  8.1417e-02,
            2.0212e-01,  1.3876e-01],
          [ 7.0346e-03, -2.8753e-02,  1.8128e-01,  ...,  3.7883e-02,
            2.1674e-01,  1.6153e-01],
          [ 5.6961e-02, -1.9288e-02,  1.3597e-01,  ...,  4.0236e-02,
            1.6788e-01,  2.1915e-01],
          ...,
   

tensor([[[[-8.5760e-02, -7.2667e-02, -3.8355e-02,  ..., -1.1798e-01,
           -1.8745e-01, -7.3636e-02],
          [-4.3518e-02, -1.5025e-01, -4.0240e-02,  ...,  8.7386e-02,
           -8.7735e-02, -1.0062e-01],
          [-5.7100e-02, -1.2626e-01, -1.6464e-01,  ..., -1.1145e-01,
           -2.0583e-01, -1.4886e-01],
          ...,
          [ 2.7739e-02, -9.5253e-02, -1.5110e-02,  ...,  4.2146e-02,
            1.4484e-01, -9.5103e-02],
          [ 4.4268e-02,  3.8773e-02,  6.5303e-03,  ...,  8.5995e-02,
            1.0288e-03, -3.8695e-01],
          [ 2.1784e-02, -2.4966e-02,  3.2689e-02,  ..., -2.0090e-01,
           -2.9549e-01, -3.8388e-01]]],


        [[[ 7.3549e-02,  1.1144e-01, -4.5958e-02,  ...,  9.3194e-02,
            1.1714e-01,  8.7869e-02],
          [ 8.5490e-02,  1.8129e-01,  7.8716e-02,  ..., -1.0158e-02,
           -2.0392e-02, -4.9577e-03],
          [-9.8808e-02, -3.8855e-02,  3.0305e-02,  ..., -2.9433e-02,
           -3.5621e-02,  4.4486e-04],
          ...,
   

tensor([[[[ 0.2161,  0.1542,  0.2111,  ..., -0.2587, -0.0710, -0.3087],
          [ 0.2228,  0.0258,  0.1233,  ..., -0.0919, -0.0260, -0.1853],
          [ 0.2567, -0.0186,  0.1544,  ..., -0.0680, -0.0715, -0.1526],
          ...,
          [ 0.2173,  0.2054,  0.2092,  ..., -0.0426, -0.2908, -0.1457],
          [ 0.2469,  0.2591,  0.2081,  ..., -0.1545, -0.3713, -0.1784],
          [ 0.1724,  0.2341,  0.1623,  ..., -0.1873, -0.3459, -0.2015]]],


        [[[ 0.1226,  0.1366,  0.1135,  ..., -0.0320, -0.0722, -0.3263],
          [ 0.1281,  0.0007,  0.0580,  ...,  0.0534, -0.1299, -0.2753],
          [ 0.2864,  0.1582,  0.2695,  ..., -0.1805, -0.1488, -0.2496],
          ...,
          [ 0.0112,  0.0358,  0.1055,  ..., -0.0667,  0.0966, -0.1734],
          [ 0.2762,  0.1632,  0.1492,  ..., -0.1139, -0.1548, -0.2889],
          [ 0.1778,  0.3341,  0.2857,  ..., -0.1591, -0.1399, -0.1737]]],


        [[[ 0.1005, -0.0935, -0.0854,  ..., -0.2335, -0.2146, -0.1780],
          [-0.0410, -0.187

tensor([[[[-0.0166,  0.0143,  0.1235,  ...,  0.3499,  0.3638,  0.2712],
          [ 0.1262,  0.0186,  0.1856,  ...,  0.3958,  0.4029,  0.3177],
          [ 0.2186,  0.1915,  0.2051,  ...,  0.2676,  0.3488,  0.2932],
          ...,
          [ 0.1553,  0.1950,  0.1729,  ...,  0.2576,  0.3462,  0.2407],
          [ 0.2726,  0.3016,  0.5144,  ...,  0.4913,  0.3294,  0.2091],
          [ 0.2372,  0.1269,  0.3794,  ...,  0.2994,  0.4238,  0.3794]]],


        [[[-0.0899, -0.0646, -0.0434,  ...,  0.2852,  0.2074,  0.0987],
          [-0.0619, -0.0334,  0.0067,  ...,  0.1574,  0.2113,  0.1079],
          [ 0.1009,  0.1792,  0.1653,  ...,  0.2091,  0.2076,  0.1592],
          ...,
          [ 0.0247,  0.0949,  0.0704,  ...,  0.1710,  0.2556,  0.2995],
          [ 0.0215,  0.0947,  0.1055,  ...,  0.2421,  0.1646,  0.1338],
          [ 0.1093,  0.1181,  0.1394,  ...,  0.2932,  0.3445,  0.4687]]],


        [[[ 0.0725,  0.0833,  0.1709,  ...,  0.3312,  0.3933,  0.3526],
          [ 0.1332,  0.122

...

tensor([[[[ 6.6454e-01,  3.7579e-01,  4.0848e-01,  ..., -5.5877e-03,
           -3.7377e-02, -3.6598e-02],
          [ 5.1329e-01,  4.5576e-01,  4.8434e-01,  ..., -1.3121e-01,
           -1.1351e-01, -1.8788e-01],
          [ 3.4563e-01,  4.3283e-01,  4.6672e-01,  ..., -5.5330e-03,
           -5.1043e-02, -2.9426e-01],
          ...,
          [ 2.0422e-01,  3.3477e-01,  2.5509e-01,  ...,  1.2490e-01,
           -9.7684e-02, -1.6988e-02],
          [ 1.5849e-01,  2.7274e-01,  2.1535e-01,  ...,  2.1905e-01,
            1.2447e-02,  8.8265e-02],
          [ 1.9930e-01,  2.8201e-01,  1.9193e-01,  ...,  1.8688e-01,
           -9.0097e-02,  3.2654e-02]]],


        [[[ 3.7035e-01,  4.7276e-01,  4.1478e-01,  ...,  2.7018e-02,
           -3.5132e-02, -1.8156e-01],
          [ 3.4697e-01,  3.9482e-01,  3.4208e-01,  ...,  8.5147e-02,
           -4.5081e-02, -1.7395e-01],
          [ 3.3320e-01,  3.0646e-01,  2.9040e-01,  ...,  1.4864e-01,
           -1.0659e-01, -1.2102e-01],
          ...,
   

tensor([[[[ 0.6003,  0.7294,  0.7535,  ...,  1.1854,  1.1491,  1.1482],
          [ 0.6781,  0.6726,  0.7784,  ...,  1.3265,  1.1564,  1.1876],
          [ 0.7858,  0.6587,  0.7442,  ...,  1.2404,  1.1160,  1.1338],
          ...,
          [ 0.6247,  0.7528,  0.7787,  ...,  1.1915,  1.2446,  1.1627],
          [ 0.5625,  0.6848,  0.8038,  ...,  1.2325,  1.2405,  1.1134],
          [ 0.5835,  0.6617,  0.8300,  ...,  1.1823,  1.1011,  1.0908]]],


        [[[ 0.5394,  0.6268,  0.8119,  ...,  0.9070,  0.9744,  1.0058],
          [ 0.5290,  0.6406,  0.7941,  ...,  0.9709,  1.0509,  1.0817],
          [ 0.3835,  0.5310,  0.5909,  ...,  1.0576,  1.0345,  1.0063],
          ...,
          [ 0.3841,  0.3766,  0.5406,  ...,  0.9302,  0.9035,  0.8577],
          [ 0.3884,  0.4851,  0.5488,  ...,  0.9882,  0.8628,  0.9079],
          [ 0.3728,  0.4984,  0.5881,  ...,  0.8495,  0.7133,  0.8775]]],


        [[[ 0.0621,  0.1355,  0.1949,  ...,  0.2428,  0.4256,  0.4471],
          [ 0.2284,  0.195

tensor([[[[-0.0687, -0.0181,  0.0483,  ...,  0.0913,  0.1395,  0.0683],
          [ 0.0848,  0.1098,  0.1851,  ...,  0.2079,  0.1964,  0.0631],
          [ 0.1485,  0.2038,  0.1800,  ...,  0.0780,  0.0787,  0.0538],
          ...,
          [ 0.1198,  0.1298,  0.2093,  ...,  0.2769,  0.2921,  0.1856],
          [ 0.0887,  0.1565,  0.1115,  ...,  0.1862,  0.2310,  0.1411],
          [ 0.1207,  0.1313,  0.1165,  ...,  0.1042,  0.0945,  0.0448]]],


        [[[ 0.3419,  0.3400,  0.3112,  ...,  0.3788,  0.2790,  0.3272],
          [ 0.2111,  0.2493,  0.2230,  ...,  0.2709,  0.2089,  0.3259],
          [ 0.0496,  0.0540,  0.0237,  ...,  0.2860,  0.2727,  0.3614],
          ...,
          [ 0.4079,  0.4090,  0.3714,  ...,  0.5850,  0.6349,  0.5361],
          [ 0.2758,  0.5095,  0.6210,  ...,  0.6994,  0.6985,  0.4348],
          [ 0.2821,  0.3578,  0.4142,  ...,  0.4282,  0.4381,  0.3309]]],


        [[[ 0.1221,  0.1065,  0.0384,  ...,  0.2871,  0.2708,  0.2437],
          [ 0.1541,  0.095

tensor([[[[-0.1804, -0.1065, -0.0582,  ..., -0.0590, -0.0818, -0.0965],
          [-0.1533, -0.1212, -0.0258,  ..., -0.0399, -0.0269, -0.1983],
          [-0.2327, -0.1446, -0.0218,  ...,  0.0273,  0.0249, -0.0683],
          ...,
          [ 0.3641,  0.2170,  0.0725,  ...,  0.0852,  0.1789,  0.1717],
          [ 0.2253,  0.1910,  0.2493,  ...,  0.1221,  0.0390,  0.0608],
          [ 0.3017,  0.3692,  0.3890,  ...,  0.1322,  0.1107,  0.1570]]],


        [[[-0.1259, -0.1616, -0.2794,  ..., -0.1767, -0.2516, -0.2144],
          [-0.0408, -0.2399, -0.3262,  ..., -0.1779, -0.2789, -0.2857],
          [-0.1280, -0.2815, -0.3593,  ..., -0.1602, -0.2606, -0.2802],
          ...,
          [-0.2368, -0.3652, -0.4388,  ..., -0.1767, -0.2744, -0.2853],
          [-0.2454, -0.3521, -0.4217,  ..., -0.2119, -0.2273, -0.2673],
          [-0.2547, -0.4028, -0.4911,  ..., -0.3108, -0.2685, -0.3340]]],


        [[[ 0.0687, -0.0942, -0.1939,  ..., -0.3016, -0.2672, -0.2372],
          [ 0.0615, -0.129

tensor([[[[-0.2452, -0.1495, -0.1319,  ..., -0.1185, -0.1696, -0.0788],
          [-0.1132,  0.0203, -0.0655,  ..., -0.0873, -0.0881, -0.0438],
          [ 0.1443,  0.1981,  0.1762,  ..., -0.0945, -0.0956, -0.0555],
          ...,
          [-0.0518,  0.1070,  0.1835,  ..., -0.0897, -0.0152, -0.0541],
          [ 0.1140,  0.1999,  0.2491,  ...,  0.0748, -0.0156, -0.0413],
          [ 0.1521,  0.1735,  0.1647,  ...,  0.1184, -0.0078, -0.1757]]],


        [[[-0.0824, -0.0634, -0.1817,  ...,  0.0638, -0.0548, -0.0184],
          [-0.0097, -0.1510, -0.3476,  ...,  0.1337,  0.0307, -0.0207],
          [ 0.0349, -0.0328, -0.2055,  ...,  0.1149,  0.0727, -0.0409],
          ...,
          [-0.0124, -0.1568, -0.1272,  ...,  0.0697,  0.1781,  0.1248],
          [ 0.0178, -0.1371, -0.0214,  ...,  0.1499,  0.1808,  0.0803],
          [-0.0533, -0.2340, -0.0660,  ...,  0.0608,  0.1621,  0.1231]]],


        [[[-0.1722, -0.2431, -0.1499,  ..., -0.2513, -0.4092, -0.3253],
          [-0.1895, -0.230

In [17]:
print(batch.shape) # 96 pieces of data, 1 channel: checking if can perform on entire batch simultaneously


torch.Size([96, 1, 28, 28])


In [18]:
# Checking if convolution can be performed on Fashion MNIST data
for batch, label in train_loader:
    convoluted = torch.nn.Conv2d(1, 1, kernel_size = (14,14), bias = False)(batch)
    print(convoluted.shape) # Input and output channels both refer to RGB channels, either way dimensionality is maintained across batches
    break

torch.Size([256, 1, 15, 15])


In [19]:
print(batch.shape) 

torch.Size([256, 1, 28, 28])


In [20]:
# Applying init to model to initialize all layer weights
def build_convnet():
    # Kernels are iterating across all three dimensions and all batch sizes simultaneously
    model = ConvNet()
    trainer = torch.optim.Adam(model.parameters(), lr = 0.005)
    loss = torch.nn.CrossEntropyLoss()
    return model, trainer, loss

In [21]:
model, optimizer, cost = build_convnet()


In [24]:
def train_model(model: torch.nn.Module, optimizer, cost, train_loader, epochs: int = 5):
    for epoch in range(epochs):
        for data, labels, in train_loader: # Returns data, label tuple
            optimizer.zero_grad()
            
            # Obtaining cross entropy cost
            loss = cost(model(data), labels)
            
            # Displaying for first epoch
            if epoch == 0:
                print("cost: ", loss)
                
            # Resetting gradient
            # Computing gradients
            loss.backward()
            # Displaying cost every 10 iterations
            optimizer.step()
        
        # Printing cost every 10th epoch
        print("cost: ", loss)

In [25]:
train_model(model, optimizer, cost, train_loader, 5) # Accuracy a little off but good initial improvement

cost:  tensor(1.7147, grad_fn=<NllLossBackward>)
cost:  tensor(1.7505, grad_fn=<NllLossBackward>)
cost:  tensor(1.7711, grad_fn=<NllLossBackward>)
cost:  tensor(1.8028, grad_fn=<NllLossBackward>)
cost:  tensor(1.8019, grad_fn=<NllLossBackward>)
cost:  tensor(1.8010, grad_fn=<NllLossBackward>)
cost:  tensor(1.7756, grad_fn=<NllLossBackward>)
cost:  tensor(1.7538, grad_fn=<NllLossBackward>)
cost:  tensor(1.7883, grad_fn=<NllLossBackward>)
cost:  tensor(1.7580, grad_fn=<NllLossBackward>)
cost:  tensor(1.7539, grad_fn=<NllLossBackward>)
cost:  tensor(1.7528, grad_fn=<NllLossBackward>)
cost:  tensor(1.7282, grad_fn=<NllLossBackward>)
cost:  tensor(1.7870, grad_fn=<NllLossBackward>)
cost:  tensor(1.7920, grad_fn=<NllLossBackward>)
cost:  tensor(1.7716, grad_fn=<NllLossBackward>)
cost:  tensor(1.7465, grad_fn=<NllLossBackward>)
cost:  tensor(1.8352, grad_fn=<NllLossBackward>)
cost:  tensor(1.7557, grad_fn=<NllLossBackward>)
cost:  tensor(1.7487, grad_fn=<NllLossBackward>)
cost:  tensor(1.8024

cost:  tensor(1.7987, grad_fn=<NllLossBackward>)
cost:  tensor(1.7371, grad_fn=<NllLossBackward>)
cost:  tensor(1.8143, grad_fn=<NllLossBackward>)
cost:  tensor(1.7520, grad_fn=<NllLossBackward>)
cost:  tensor(1.7575, grad_fn=<NllLossBackward>)
cost:  tensor(1.7517, grad_fn=<NllLossBackward>)
cost:  tensor(1.7722, grad_fn=<NllLossBackward>)
cost:  tensor(1.7396, grad_fn=<NllLossBackward>)
cost:  tensor(1.7111, grad_fn=<NllLossBackward>)
cost:  tensor(1.8045, grad_fn=<NllLossBackward>)
cost:  tensor(1.7311, grad_fn=<NllLossBackward>)
cost:  tensor(1.7390, grad_fn=<NllLossBackward>)
cost:  tensor(1.7727, grad_fn=<NllLossBackward>)
cost:  tensor(1.7360, grad_fn=<NllLossBackward>)
cost:  tensor(1.7539, grad_fn=<NllLossBackward>)
cost:  tensor(1.8146, grad_fn=<NllLossBackward>)
cost:  tensor(1.7631, grad_fn=<NllLossBackward>)
cost:  tensor(1.7541, grad_fn=<NllLossBackward>)
cost:  tensor(1.7498, grad_fn=<NllLossBackward>)
cost:  tensor(1.7698, grad_fn=<NllLossBackward>)
cost:  tensor(1.7455

Rigorously, a convolution is left shifted instead of right shifted, rendering this a cross correlation. However, since either way our kernels are learned, there is no difference made in the learning pattern - the kernel transposed would be identical and an identical gradient is applied to the transposed kernel

**Feature Map** - The output of a convolutional layers, as it is a set of localized aggregated features in space.

**Receptive Field** - The set of all elements which affect the computation of a feature. A feature with a large receptive field is a function of a large portion of the image and would represent a pattern taking place in a large portion of the image.